Imports.

In [ ]:
import sys
import warnings
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd

In [ ]:
D_INDICES = dict(voltage_index = 2,
                 current_index = 3,
                 capacity_index = 5,
                 date_index = -2,
                 time_index = -1,
                 )

Ignore UserWarnings in case `sys.exit()` is encountered.

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

Checking whether a folder called `data` exists.

In [ ]:
data_path = Path.cwd() / "data"
if not data_path.exists():
    data_path.mkdir()
    s = f"\n{80*'-'}\nA folder called '{data_path.name}' has been created."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Checking that data files are present in the `data` folder.

In [ ]:
data_files = list(data_path.glob("*.*"))
if len(data_files) == 0:
    s = f"\n{80*'-'}\nNo files were found in the '{data_path.name}' folder.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Function to get header and index for header line in the data file.

In [ ]:
def mti_extract(file, d_indices):
    with file.open(mode="r") as f:
        lines = f.readlines()
    header = lines[0:3]
    lines = lines[3::]
    modes = ["Rest", "CC_Chg", "CC_DChg"]
    start = None
    for i, line in enumerate(lines):
        line = line.split()
        for e in modes:
            if e in line:
                start = i + 1
                break
        if not isinstance(start, type(None)):
            break
    cols = len(lines[start].split())
    cols_cycles = len(lines[start-2].split())
    time_cycle_start = []
    for i in range(len(lines)):
        line_split = lines[i].split()
        if len(line_split) == cols_cycles:
            line_split_data = lines[i+2].split()
            date_split = line_split_data[d_indices["date_index"]].split("-")
            time_split = line_split_data[d_indices["time_index"]].split(":")
            time_cycle_start.append(datetime(int(date_split[0]),
                                             int(date_split[1]),
                                             int(date_split[2]),
                                             int(time_split[0]),
                                             int(time_split[1]),
                                             int(time_split[2])
                                             ).timestamp()
                                    )
    time_cycle_start = np.array(time_cycle_start)
    time_cycle_start = time_cycle_start - time_cycle_start[0]
    voltage, current, capacity, date, time, date_time = [], [], [], [], [], []
    for i in range(start, len(lines)):
        line = lines[i].split()
        if len(line) == cols:
            voltage.append(float(line[d_indices["voltage_index"]]))
            current.append(float(line[d_indices["current_index"]]))
            capacity.append(float(line[d_indices["capacity_index"]]))
            date.append(line[d_indices["date_index"]])
            time.append(line[d_indices["time_index"]])
    for i in range(len(date)):
        date_split = date[i].split("-")
        time_split = time[i].split(":")
        date_time.append(datetime(int(date_split[0]),
                                  int(date_split[1]),
                                  int(date_split[2]),
                                  int(time_split[0]),
                                  int(time_split[1]),
                                  int(time_split[2])
                                  ).timestamp()
                        )
    voltage, current = np.array(voltage), np.array(current) * 10**-3
    capacity = np.array(capacity)
    date_time = np.array(date_time)
    time = date_time - date_time[0]
    df = pd.DataFrame(dict(time=time,
                           voltage=voltage,
                           current=current,
                        #    capacity=capacity,
                        #    time_cycle_start=time_cycle_start,
                           ))

    return df

Function to write the extracted data to a `.txt` file.

In [ ]:
def write(df, filename, output_path):
    time = df["time"].to_numpy() / 60**2
    voltage, current = df["voltage"].to_numpy(), df["current"].to_numpy()
    header = "t [h]\tV [V]\ti [A]"
    np.savetxt(output_path / filename,
               np.column_stack((time, voltage, current)),
               delimiter="\t",
               header=header,
               fmt="%.8f",
               encoding="utf8",
               )
    
    return None

Making sure that the output path exists.

In [ ]:
txt_path = Path.cwd() / "txt"
if not txt_path.exists():
    txt_path.mkdir()

Extracting data and writing to `.txt` file.

In [ ]:
print(f"{80*'-'}\nWorking with files...")
for f in data_files:
      print(f"\t{f.name}")
      for f in data_files:
            df = mti_extract(f, D_INDICES)
      write(df, f.name, txt_path)
print(f"Done working with files. Please see the '{txt_path.name}' folder."
      f"\n{80*'-'}")